<a href="https://colab.research.google.com/github/prasann25/colab/blob/main/06_transfer_learning_in_tensorflow-part3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning With TensorFlow Part 3: Scaling Up (Food Vision mini)

We've seen the power of transfer learning feature extraction and fine-tuning, now it's time to scale up to all of the classes in Food101(101 total classes of food).

Our goal is to beat the original Food101 paper with 10% of the training ( leveraging the power of deep learning).

Original Food101 paper : https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/static/bossard_eccv14_food-101.pdf

Our baseline to beat is 50.76% accuracy across 101 classes.

In [1]:
# Check to see if we're using a GPU
!nvidia-smi

Mon Jul 19 03:25:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions
In previous notebooks, we've created a series of helper functions to do different tasks, let's download them



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-07-19 03:25:46--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-19 03:25:47 (90.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper funcitons for our notebook
from helper_functions import plot_loss_curves, create_tensorboard_callback, unzip_data, compare_historys, walk_through_dir

## 101 Food Classes: working with less data
Our goal is to beat the original Food101 paper with 10% of the training data, so let's download it.

The data we're downloading comes from the original Food101 dataset but has been preprocessed using the `image_data_modification` notebook - https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/image_data_modification.ipynb

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")

--2021-07-19 03:25:49--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 142.250.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  78.6MB/s    in 9.6s    

2021-07-19 03:25:59 (161 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [ ]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"


In [ ]:
# How many images/classes are there?
walk_through_dir(dir_path="101_food_classes_10_percent")

In [ ]:
# Setup data inputs
import tensorflow as tf

IMG_SIZE= (224, 224)
BATCH_SIZE=32
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                image_size=IMG_SIZE,
                                                                                batch_size=BATCH_SIZE,
                                                                                label_mode="categorical")
# Not Shuffle the test data for prediction analysis
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                batch_size=BATCH_SIZE,
                                                                shuffle=False) # don't shuffle test data for prediction analysis

## Train a big dog model with transfer learning on 10% of 101 food classes
Here are the steps we're going to take :
* Create a ModelCheckpoint callback
* Create data augmentation layer to build data augmentation right into model
* Build a headless (no top layers) Functional EfficientNetB0 backboned-model ( we'll create our own output layer)
* Compile our model
* Feature extract for 5 full passes (5 epochs on the train dataset and validate on 15% of the test data, to save epoch time)


In [ ]:
# Create  checkpoint callback

checkpoint_path="101_classes_10_percent_data_model_checkpoint"
checkpoint_callback  = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,                                                          
                                                          save_weights_only=True,
                                                          monitor="val_accuracy",
                                                          save_best_only=True,
                                                          save_freq="epoch")


In [ ]:
# Create data augmentation layer to incorporate it right into the model

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentation = Sequential([
              preprocessing.RandomFlip("horizontal"),
              preprocessing.RandomRotation(0.2),
              preprocessing.RandomHeight(0.2),
              preprocessing.RandomWidth(0.2),
              preprocessing.RandomZoom(0.2),
              #preprocessing.Rescaling(1/255.), # rescale inputs of images to between 0 & 1, required for model like ResNet50              
            ], name="data_augmentation")

In [ ]:
# Setup the base model and freeze its layers(this will extract features)

base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup  model architectures with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")

x = data_augmentation(inputs) # augment images (only happens during the training phase)

x = base_model(x, training = False) # put the base model in inference mode, so weights which need to stay frozen, stay frozen

x = layers.GlobalAvgPool2D(name="global_avg_pool_layer")(x)

# For layers, the inputs or x goes outside. For model it can go inside 
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer") (x)

model = tf.keras.Model(inputs, outputs)


In [ ]:
# Get a summary of model we've created
model.summary()

In [ ]:
# Compile our model
model.compile(loss="categorical_crossentropy",
             optimizer=tf.keras.optimizers.Adam(),
             metrics=["accuracy"])

history_all_classes_10_percent = model.fit(train_data_all_10_percent, 
                                           epochs=5, # fit for epochs to keep experiment quick
                                           validation_data = test_data,
                                           validation_steps = int(0.15 *len(test_data)), # validate on only 15% of the data during training
                                           callbacks=[checkpoint_callback])

In [ ]:
# Evaluate the model
model.evaluate(test_data)


In [ ]:
# Plot loss curves
plot_loss_curves(history_all_classes_10_percent)

In [ ]:
> **Question** : What do these curves suggest? 
Note: Our model is overfitting as seen above, val_loss and val_accuracy deviating from training curves.